<a href="https://colab.research.google.com/github/M-kaur07/Hardware_Trojan_Detection_GNN/blob/main/Scoap_RS232.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving RS232-T1100.zip to RS232-T1100.zip
Saving RS232-T1000.zip to RS232-T1000.zip


In [ ]:
import zipfile

# Unzip both benchmark files
with zipfile.ZipFile("RS232-T1000.zip", 'r') as zip_ref:
    zip_ref.extractall("RS232-T1000")

with zipfile.ZipFile("RS232-T1100.zip", 'r') as zip_ref:
    zip_ref.extractall("RS232-T1100")

print("✅ Files extracted successfully.")


FileNotFoundError: [Errno 2] No such file or directory: 'RS232-T1000.zip'

In [ ]:
!pip install pyverilog networkx matplotlib


In [ ]:
import re

def manual_gate_parser(verilog_path):
    with open(verilog_path, 'r') as file:
        lines = file.readlines()

    gate_list = []
    for line in lines:
        line = line.strip()
        if line.startswith('//') or line == '' or not ('(' in line and ')' in line):
            continue
        if any(keyword in line for keyword in ['module', 'endmodule', 'input', 'output', 'wire']):
            continue

        tokens = line.split()
        if len(tokens) < 2:
            continue
        gate_type = tokens[0]
        gate_name = tokens[1]
        ports_raw = line[line.find('(')+1 : line.rfind(')')]
        ports = ports_raw.split(',')
        conn_dict = {}
        for port in ports:
            if ('(' in port and ')' in port) or port.count('.') >= 1:
                parts = port.strip().replace(')', '').replace('(', '').replace(';', '').split('.')
                if len(parts) >= 2:
                    conn_dict[parts[0].strip()] = parts[1].strip()
        gate_list.append({
            'gate_type': gate_type,
            'gate_name': gate_name,
            'connections': conn_dict
        })
    return gate_list


In [ ]:
import networkx as nx

def build_graph(gate_list):
    G = nx.DiGraph()
    for gate in gate_list:
        gate_node = gate['gate_name']
        G.add_node(gate_node, gate_type=gate['gate_type'])
        for pin, net in gate['connections'].items():
            G.add_edge(net, gate_node, pin=pin)
            G.add_edge(gate_node, net, pin=pin)
    return G


In [ ]:
'''def compute_scoap(G):
    cc0 = {}
    cc1 = {}
    for node in nx.topological_sort(G):
        preds = list(G.predecessors(node))
        if not preds:
            cc0[node] = 1
            cc1[node] = 1
        else:
            cc0[node] = min([cc0.get(p, 1) for p in preds]) + 1
            cc1[node] = min([cc1.get(p, 1) for p in preds]) + 1
    return cc0, cc1
'''

def compute_scoap(G):
    cc0 = {}
    cc1 = {}

    for node in nx.topological_sort(G):
        gate_type = G.nodes[node].get('gate_type', '').upper()
        preds = [p for p in G.predecessors(node) if p in cc0]  # avoid signal nodes
        if not preds:
            cc0[node] = 1
            cc1[node] = 1
        else:
            if gate_type == 'AND':
                cc0[node] = min(cc0[p] for p in preds) + 1
                cc1[node] = sum(cc1[p] for p in preds) + 1
            elif gate_type == 'OR':
                cc0[node] = sum(cc0[p] for p in preds) + 1
                cc1[node] = min(cc1[p] for p in preds) + 1
            elif gate_type == 'NAND':
                cc0[node] = min(cc0[p] for p in preds) + 1
                cc1[node] = sum(cc1[p] for p in preds) + 1
            elif gate_type == 'NOR':
                cc0[node] = sum(cc0[p] for p in preds) + 1
                cc1[node] = min(cc1[p] for p in preds) + 1
            elif gate_type == 'NOT':
                cc0[node] = cc1[preds[0]] + 1
                cc1[node] = cc0[preds[0]] + 1
            elif gate_type in ['BUF', 'BUFF', 'BUFFER']:
                cc0[node] = cc0[preds[0]]
                cc1[node] = cc1[preds[0]]
            else:
                # Default fall-back: conservative
                cc0[node] = min(cc0[p] for p in preds) + 1
                cc1[node] = min(cc1[p] for p in preds) + 1

    return cc0, cc1


In [ ]:
def analyze_suspicious_gates(G, cc0, cc1, threshold=5):
    suspicious = []
    for gate in G.nodes:
        if cc0.get(gate, 0) > threshold and cc1.get(gate, 0) > threshold:
            suspicious.append((gate, cc0[gate], cc1[gate]))
    return suspicious

In [ ]:
import os

for root, dirs, files in os.walk("."):
    for file in files:
        if file.endswith(".v"):
            print(os.path.join(root, file))


In [ ]:
# Paths to Verilog netlists
trojan1_path = "./RS232-T1000/RS232-T1000/src/90nm/uart.v"
trojan11_path = "./RS232-T1100/RS232-T1100/src/90nm/uart.v"


In [ ]:
def manual_gate_parser(verilog_path):
    gates = []
    with open(verilog_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith("//") or line.startswith("`") or len(line) < 3:
                continue
            if ";" in line and "(" in line and ")" in line:
                try:
                    gate_type = line.split()[0]
                    gate_name = line.split()[1]
                    connections_str = line[line.find("(")+1:line.find(")")]
                    conns = connections_str.split(",")
                    conn_dict = {}
                    for c in conns:
                        pin_net = c.strip().split("(")
                        if len(pin_net) == 2:
                            pin, net = pin_net
                            conn_dict[pin.strip()] = net.replace(")", "").strip()
                    gates.append({
                        'gate_type': gate_type,
                        'gate_name': gate_name,
                        'connections': conn_dict
                    })
                except:
                    pass
    return gates


In [ ]:
'''import networkx as nx

def build_graph(gates):
    G = nx.DiGraph()
    for gate in gates:
        gate_name = gate['gate_name']
        G.add_node(gate_name, gate_type=gate['gate_type'], connections=gate['connections'])

        for pin, net in gate['connections'].items():
            if pin.startswith("IN") or pin in ["A", "B", "C", "D"]:
                for other_gate in gates:
                    if net in other_gate['connections'].values():
                        driver = other_gate['gate_name']
                        if driver != gate_name:
                            G.add_edge(driver, gate_name)
    return G'''

import networkx as nx

def build_graph(gates):
    G = nx.DiGraph()
    net_drivers = {}  # net → gate that drives it

    # Step 1: Add all gates as nodes and track output nets
    for gate in gates:
        gate_name = gate['gate_name']
        gate_type = gate['gate_type']
        G.add_node(gate_name, gate_type=gate_type, connections=gate['connections'])

        for pin, net in gate['connections'].items():
            # Heuristic: output pins are often named Y, Z, O, OUT, etc.
            if pin.upper() in ['Y', 'Z', 'O', 'OUT', 'Q']:
                net_drivers[net] = gate_name

    # Step 2: Create edges from driver to consumer gates
    for gate in gates:
        gate_name = gate['gate_name']
        for pin, net in gate['connections'].items():
            if pin.upper() not in ['Y', 'Z', 'O', 'OUT', 'Q']:  # assume input
                driver = net_drivers.get(net)
                if driver and driver != gate_name:
                    G.add_edge(driver, gate_name, net=net, pin=pin)

    return G



In [ ]:
'''def compute_scoap(G):
    cc0 = {}
    cc1 = {}

    try:
        topo = list(nx.topological_sort(G))
    except nx.NetworkXUnfeasible:
        print("Graph has cycles. Breaking all cycles to form DAG.")
        G = G.copy()
        while True:
            try:
                cycle = nx.find_cycle(G, orientation='original')
                G.remove_edge(*cycle[0][:2])  # remove one edge from the cycle
            except nx.NetworkXNoCycle:
                break
        topo = list(nx.topological_sort(G))

    for node in topo:
        preds = list(G.predecessors(node))
        if not preds:
            cc0[node] = 1
            cc1[node] = 1
        else:
            cc0[node] = sum([cc0.get(p, 1) for p in preds]) + 1
            cc1[node] = sum([cc1.get(p, 1) for p in preds]) + 1

    return cc0, cc1'''
def compute_scoap(G):
    cc0 = {}
    cc1 = {}

    try:
        topo = list(nx.topological_sort(G))
    except nx.NetworkXUnfeasible:
        print("Graph has cycles. Breaking all cycles to form DAG.")
        G = G.copy()
        while True:
            try:
                cycle = nx.find_cycle(G, orientation='original')
                G.remove_edge(*cycle[0][:2])  # remove one edge from the cycle
            except nx.NetworkXNoCycle:
                break
        topo = list(nx.topological_sort(G))

    for node in topo:
        preds = list(G.predecessors(node))
        gate_type = G.nodes[node].get('gate_type', '').upper()

        if not preds:
            cc0[node] = 1
            cc1[node] = 1
        else:
            pred_cc0 = [cc0.get(p, 1) for p in preds]
            pred_cc1 = [cc1.get(p, 1) for p in preds]

            if gate_type == 'AND':
                cc0[node] = min(pred_cc0) + 1
                cc1[node] = sum(pred_cc1) + 1
            elif gate_type == 'OR':
                cc0[node] = sum(pred_cc0) + 1
                cc1[node] = min(pred_cc1) + 1
            elif gate_type == 'NAND':
                cc0[node] = min(pred_cc0) + 1
                cc1[node] = sum(pred_cc1) + 1
            elif gate_type == 'NOR':
                cc0[node] = sum(pred_cc0) + 1
                cc1[node] = min(pred_cc1) + 1
            elif gate_type == 'NOT':
                cc0[node] = pred_cc1[0] + 1
                cc1[node] = pred_cc0[0] + 1
            elif gate_type in ['BUF', 'BUFF', 'BUFFER']:
                cc0[node] = pred_cc0[0]
                cc1[node] = pred_cc1[0]
            else:
                # Default fallback if unknown gate
                cc0[node] = min(pred_cc0) + 1
                cc1[node] = min(pred_cc1) + 1

    return cc0, cc1



In [ ]:
def analyze_and_report(verilog_path, label):
    gates = manual_gate_parser(verilog_path)
    print(f"{label}: Parsed {len(gates)} gates")

    G = build_graph(gates)
    cc0, cc1 = compute_scoap(G)

    nx.set_node_attributes(G, cc0, "cc0")
    nx.set_node_attributes(G, cc1, "cc1")

    suspicious = [n for n in G.nodes if cc0[n] > 10 and cc1[n] > 10]
    print(f"{label}: Suspicious gates (CC0 & CC1 > 10): {len(suspicious)}")
    if suspicious:
        print("Example suspicious gates:", suspicious[:5])

    return suspicious


In [ ]:
# Step 1: Parse Verilog gates
gates = manual_gate_parser("./RS232-T1000/RS232-T1000/src/90nm/uart.v")  # or T1100

# Step 2: Build graph
G = build_graph(gates)

# Step 3: Compute SCOAP
cc0, cc1 = compute_scoap(G)


FileNotFoundError: [Errno 2] No such file or directory: './RS232-T1000/RS232-T1000/src/90nm/uart.v'

In [ ]:
# Step 1: Parse Verilog gates
gates = manual_gate_parser("./RS232-T1100/RS232-T1100/src/90nm/uart.v")

# Step 2: Build graph
G = build_graph(gates)

# Step 3: Compute SCOAP
cc0, cc1 = compute_scoap(G)


FileNotFoundError: [Errno 2] No such file or directory: './RS232-T1100/RS232-T1100/src/90nm/uart.v'

In [ ]:
suspicious_trojan1 = analyze_and_report(trojan1_path, "Trojaned RS232-T1000")
suspicious_trojan11 = analyze_and_report(trojan11_path, "Trojaned RS232-T1100")


NameError: name 'trojan1_path' is not defined

In [ ]:
suspicious = [n for n in G.nodes if cc0[n] > 5 or cc1[n] > 5]


NameError: name 'G' is not defined

In [ ]:
import matplotlib.pyplot as plt

def visualize_graph(G, suspicious_nodes):
    pos = nx.spring_layout(G, seed=42)
    color_map = ['red' if node in suspicious_nodes else 'lightblue' for node in G.nodes]
    nx.draw(G, pos, node_color=color_map, with_labels=False, node_size=50)
    plt.title("Suspicious Gates Highlighted in Red")
    plt.show()


In [ ]:
visualize_graph(G, suspicious)


NameError: name 'G' is not defined

In [ ]:
max_cc0 = max(cc0.values())
avg_cc0 = sum(cc0.values()) / len(cc0)

print(f"Max CC0: {max_cc0}, Avg: {avg_cc0:.2f}")


NameError: name 'cc0' is not defined

In [ ]:
def compute_observability(G):
    co = {}

    # Step 1: Identify primary outputs (nodes with no successors)
    primary_outputs = [n for n in G.nodes if G.out_degree(n) == 0]

    # Initialize primary outputs with CO = 1
    for node in G.nodes:
        co[node] = float('inf')  # default high cost
    for po in primary_outputs:
        co[po] = 1

    # Step 2: Reverse topological sort
    try:
        topo_rev = list(reversed(list(nx.topological_sort(G))))
    except nx.NetworkXUnfeasible:
        print("Graph has cycles. Cannot compute observability reliably.")
        return co  # fallback with default values

    # Step 3: Propagate CO from outputs to inputs
    for node in topo_rev:
        succs = list(G.successors(node))
        if not succs:
            continue  # already initialized

        min_succ_co = min([co[s] for s in succs])
        gate_type = G.nodes[node].get('gate_type', '').upper()

        if gate_type == 'AND':
            co[node] = min_succ_co + 1
        elif gate_type == 'OR':
            co[node] = min_succ_co + 1
        elif gate_type == 'NAND':
            co[node] = min_succ_co + 1
        elif gate_type == 'NOR':
            co[node] = min_succ_co + 1
        elif gate_type == 'NOT':
            co[node] = min_succ_co + 1
        elif gate_type in ['BUF', 'BUFF', 'BUFFER']:
            co[node] = min_succ_co
        else:
            # fallback for unknown types
            co[node] = min_succ_co + 1

    return co


In [ ]:
co = compute_observability(G)
nx.set_node_attributes(G, co, "co")


NameError: name 'G' is not defined

In [ ]:
suspicious_co = [n for n in G.nodes if co[n] > 5]
print("Suspicious based on CO:", suspicious_co)


NameError: name 'G' is not defined